<img src = "https://8weeksqlchallenge.com/images/case-study-designs/2.png">

## Introduction

Did you know that over 115 million kilograms of pizza is consumed daily worldwide??? (Well according to Wikipedia anyway…)

Danny was scrolling through his Instagram feed when something really caught his eye - “80s Retro Styling and Pizza Is The Future!”

Danny was sold on the idea, but he knew that pizza alone was not going to help him get seed funding to expand his new Pizza Empire - so he had one more genius idea to combine with it - he was going to Uberize it - and so Pizza Runner was launched!

## Problem Statement

Danny started by recruiting “runners” to deliver fresh pizza from Pizza Runner Headquarters (otherwise known as Danny’s house) and also maxed out his credit card to pay freelance developers to build a mobile app to accept orders from customers.


## Entity Relationship Diagram

<img src = "https://user-images.githubusercontent.com/81607668/127271531-0b4da8c7-8b24-4a14-9093-0795c4fa037e.png">

In [1]:
import sqlite3
import pandas as pd

In [2]:
database = '../data/PizzaRunner.sqlite'

In [3]:
connection = sqlite3.connect(database)

# Case Study Questions

---

## C. Ingredient Optimisation


---

### 1. What are the standard ingredients for each pizza?

In [4]:
pd.read_sql("""


        WITH RECURSIVE split(pizza_id, topping_id, rest) AS (
           SELECT pizza_id, '', toppings||',' FROM pizza_recipes
           UNION ALL SELECT
           pizza_id,
           substr(rest, 0, instr(rest, ',')),
           substr(rest, instr(rest, ',')+1)
           FROM split WHERE rest!=''
           ORDER BY pizza_id
        )

        SELECT 
        pn.pizza_name AS Pizza , pt.topping_name AS Ingredient
        FROM split AS s
        INNER JOIN pizza_names AS pn
        ON s.pizza_id = pn.pizza_id
        INNER JOIN pizza_toppings AS pt
        ON s.topping_id = pt.topping_id

            
            """,connection)

,Pizza,Ingredient
0,Meatlovers,Salami
1,Meatlovers,BBQ Sauce
2,Meatlovers,Beef
3,Meatlovers,Cheese
4,Meatlovers,Chicken
5,Meatlovers,Mushrooms
6,Meatlovers,Pepperoni
7,Meatlovers,Bacon
8,Vegetarian,Tomatoes
9,Vegetarian,Tomato Sauce


### 2. What was the most common extra?

In [5]:
pd.read_sql("""

        WITH RECURSIVE extra_split(order_id, customer_id ,pizza_id, extras, rest) AS (
            SELECT order_id, customer_id ,pizza_id, '', extras||',' FROM customer_orders
            UNION ALL SELECT
            order_id, customer_id ,pizza_id,
            substr(rest, 0, instr(rest, ',')),
            substr(rest, instr(rest, ',')+1)
            FROM extra_split WHERE rest!=''
            ORDER BY order_id, customer_id ,pizza_id
        )

            SELECT topping_name AS Extra, COUNT(extras) AS "Times ordered"
            FROM extra_split AS es
            INNER JOIN pizza_toppings AS pr
            ON es.extras = pr.topping_id
            GROUP BY topping_name
            ORDER BY "Times Ordered" DESC
            LIMIT 1;


""",connection)

,Extra,Times ordered
0,Bacon,4


### 3. What was the most common exclusion?

In [6]:
pd.read_sql("""

        WITH RECURSIVE exclusion_split(order_id, customer_id ,pizza_id, exclusion, rest) AS (
            SELECT order_id, customer_id ,pizza_id, '', exclusions||',' FROM customer_orders
            UNION ALL SELECT
            order_id, customer_id ,pizza_id,
            substr(rest, 0, instr(rest, ',')),
            substr(rest, instr(rest, ',')+1)
            FROM exclusion_split WHERE rest!=''
            ORDER BY order_id, customer_id ,pizza_id
        )

            SELECT topping_name AS Exclusion, COUNT(exclusion) AS "Times ordered"
            FROM exclusion_split AS es
            INNER JOIN pizza_toppings AS pr
            ON es.exclusion = pr.topping_id
            GROUP BY topping_name
            ORDER BY "Times Ordered" DESC
            LIMIT 1;

""",connection)

,Exclusion,Times ordered
0,Cheese,4


### 4. Generate an order item for each record in the customers_orders table in the format of one of the following:

- Meat Lovers
- Meat Lovers - Exclude Beef
- Meat Lovers - Extra Bacon
- Meat Lovers - Exclude Cheese, Bacon - Extra Mushroom, Peppers

### 5. Generate an alphabetically ordered comma separated ingredient list for each pizza order from the customer_orders table and add a 2x in front of any relevant ingredients For example: "Meat Lovers: 2xBacon, Beef, ... , Salami"

### 6. What is the total quantity of each ingredient used in all delivered pizzas sorted by most frequent first?